For each mouse
1. (Manual) Find summary excel file, extract names of green and yellow sessions
2. VidsDCIMG = Find video files, one for each trial
3. refImg = Find reference image
4. trialTypes = Find trial types
5. t1.mat, t2.mat = Find warping files 
6. allenBrain = Find allen brain map

Processing:
1. Testing_A:
    * For each mouse, load ::: refImg, t1.mat, allenBrain
      * Gallero\SDT-TDT\overlay\mou_9
        - L_modified (brain areas for all mice) ::: Gallero\SDT-TDT\overlay\mou_9/L_modified.mat
        - RefImg ::: Gallero\SDT-TDT\overlay\mou_9\refImg_ROIs.mat
    * For each session, load ::: t2.mat
      - Gallero/mou_9/TDT/20180315/widefield_labview/ROI_Allen/registration_transform_1510.mat
    * Apply transform to refImg, overplot allenBrain, check that they align
2. Testing_B:
    * Load all trial videos
        - Gallero\mou_9\TDT\20180301\widefield_labview
    * Compute time-average images
    * Apply same as Testing_A
    * Plot a few random sessions
3. Process real data - for each session:
    * Provide list of sessions to analyze
    * Provide root folder to mouse data
    * Provide root folder to overlay data
    * Videos
        * Load all trial videos
        * Apply transformation
        * Apply allen map to extract activities, average over pixels
        * Store in HDF5
    * Trial Intervals:
        * Load trial interval file
          - Gallero\mou_9\TDT\20180301\widefield_labview\a\Matt_files\20180301a.mat
        * Store 3rd column to HDF5
    * Trial Types:
        * Load trial type file
          - \Gallero\mou_9\TDT\20180301\widefield_labview\a\Matt_files/trials_ind.mat
        * Parse, store to HDF5
4. Postprocess datatypes
    * Read HDF5
    * Merge trials for each session
    * Calc times from intervals
    * Apply polyfit, look
    * Perform DFF, store


Notes:

* Matlab Fitting routines
   - t1= align to mapping day (functional); aligns every day to a common day.
   - t2= aligns the common day to Allen institue

`
tform = fitgeotrans(movingPoints,fixedPoints,'polynomial',3);
Jregistered = imwarp(refImg*7000,tform,'OutputView',imref2d(size(A_scaled))); %aligned map
`

**TODO**:
* Compare TrialTypes to to trial_idx.m where applicable, otherwise trial_error.m
* Compare resulting bn_trial with what Yasir got

In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fpath

from lib.gallerosalas.preprocess_raw import preprocess
import lib.gallerosalas.preprocess_common as prepcommon


%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis


In [2]:
FPS = 20  # Hz
mice = ['mou_9', 'mou_6', 'mou_7', 'mou_5']
extra_mice = ['mou_15_Ariel', 'mou_18_Ariel']

In [3]:
pathPrefix = '/run/user/1000/gvfs/smb-share:server=130.60.51.15,share=neurophysiology-storage2/'

In [4]:
# pathTDT = gui_fpath('root path to TDT')
# pathTDT = pathPrefix + 'Gallero'
pathTDT = pathPrefix + 'Gallero/DATA PER MICE (raw and preprocessed)/Widefield_imaging mice/'

In [5]:
# pathOverlay = gui_fpath('root path to overlay')
# pathOverlay = pathPrefix + 'SDT-TDT/overlay'
pathOverlay = pathPrefix + 'Gallero/Data_processed/SDT-TDT/overlay'

In [6]:
# pathPreferences = gui_fpath('root path to overlay')
pathPreferences = '/media/alyosha/Data/TE_data/yasirdata_raw'
# pathPreferences = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [7]:
pathDict = {
    'TDT' : pathTDT,
    'Overlay' : pathOverlay,
    'Preferences' : pathPreferences
}

In [8]:
prep = preprocess(pathDict)

Parsing TDT:
-- mou_5 2017_03_06
-- mou_5 2017_03_13
-- mou_5 2017_03_14
-- mou_5 2017_03_15
-- mou_5 2017_03_16
-- mou_5 2017_03_17
-- mou_5 2017_03_22
-- mou_5 2017_03_23
-- mou_5 2017_03_24
-- mou_5 2017_03_28
-- mou_5 2017_03_29
-- mou_6 20170925
-- mou_6 20170926
-- mou_6 20170927
-- mou_6 20170928
-- mou_6 20170929
-- mou_6 20171002
-- mou_6 20171004
-- mou_6 20171005
-- mou_6 20171006
-- mou_6 20171009
-- mou_6 20171010
-- mou_6 20171011
-- mou_6 20171012
-- mou_7 20171113
-- mou_7 20171114
-- mou_7 20171115
-- mou_7 20171116
-- mou_7 20171122
-- mou_7 20171123
-- mou_7 20171127
-- mou_7 20171128
-- mou_7 20171129
-- mou_7 20171130
-- mou_9 20180301
-- mou_9 20180302
-- mou_9 20180305
-- mou_9 20180306
-- mou_9 20180307
-- mou_9 20180308
-- mou_9 20180309
-- mou_9 20180313
-- mou_9 20180315
-- mou_9 20180316
Parsing overlay:
-- mou_5
-- mou_6
-- mou_7
-- mou_9


# 1. Test transform on Ref files

In [ ]:
for mouseName, pathRef in prep.pathRef.items():
    img = prep.load_ref_img(pathRef)
    t2 = prep.load_t2(prep.pathT2[mouseName])
    print(mouseName)
    
    imgT2 = prep.transform_img(img, t2)[0]
    prep.plot_transforms(img, imgT2, imgT1=None)

# 2. Test transform on Video Files

In [ ]:
%%time
for mousename in sorted(set(prep.dataPaths['mouse'])):
    print(mousename)
    prep.test_transform_vids(mousename)

# 3. Pool-process video files

In [ ]:
prep.process_video_files('mou_5', skipExisting=False)

## 3.1 Test if result is sensible

In [ ]:
import h5py 
rez = []
for idx, row in prep.dataPaths.iterrows():
    session = row['day'] + '_' + row['session']
    fpath = os.path.join(pathPreferences, row['mouse'] + '.h5')    
    
    with h5py.File(fpath, 'r') as f:
        dataRSP = np.copy(f['data'][session])
        rez += [np.nanmean(dataRSP, axis=(0, 1))]
        
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(np.array(rez), columns=np.arange(27)))

# 4. Pull trial types and times

* Also compute d'

In [27]:
import h5py

for mouseName in mice:
    with h5py.File(os.path.join(pathPreferences, mouseName + '.h5'), 'a') as f:
        for key in ['metadataTrial', 'metadataSession', 'accuracy', 'dprime']:
            if key in f.keys():
                del f[key]

In [28]:
prep.process_metadata_files(pathPreferences)

mou_5 2017_03_06_session01
2017-03-06 14:58:05.031300
{'Hit': 89, 'Miss': 0, 'CR': 69, 'FA': 14}
mou_5 2017_03_06_session02
2017-03-06 16:05:45.684500
{'Hit': 43, 'Miss': 13, 'CR': 50, 'FA': 2}
mou_5 2017_03_13_session01
2017-03-13 16:56:50.137800
{'Hit': 74, 'Miss': 9, 'CR': 83, 'FA': 4}
mou_5 2017_03_13_session02
2017-03-13 17:55:49.346300
{'Hit': 50, 'Miss': 9, 'CR': 56, 'FA': 3}
mou_5 2017_03_14_session02
2017-03-14 15:32:35.633600
{'Hit': 71, 'Miss': 52, 'CR': 84, 'FA': 17}
mou_5 2017_03_15_session01
2017-03-15 12:16:52.368300
{'Hit': 53, 'Miss': 4, 'CR': 50, 'FA': 19}
mou_5 2017_03_16_session01
2017-03-16 13:45:04.029900


/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '2017_03_16_session01'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


{'Hit': 58, 'Miss': 1, 'CR': 45, 'FA': 20}
mou_5 2017_03_16_session02
2017-03-16 15:14:29.787600


/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '2017_03_16_session02'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


{'Hit': 47, 'Miss': 2, 'CR': 47, 'FA': 11}
mou_5 2017_03_16_session03
2017-03-16 16:05:36.600000


...

/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '20180315_a'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


{'Hit': 65, 'Miss': 12, 'CR': 65, 'FA': 22}
mou_9 20180316_a
2018-03-16 08:27:55.893500


/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '20180316_a'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


{'Hit': 93, 'Miss': 11, 'CR': 83, 'FA': 14}
mou_9 20180316_b
2018-03-16 09:29:45.654400
{'Hit': 43, 'Miss': 3, 'CR': 47, 'FA': 3}


/opt/anaconda3/envs/py3ml/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '20180316_b'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [29]:
for idx, row in prep.dataPaths.iterrows():
    session = row['day'] + '_' + row['session']
    fpath = os.path.join(pathPreferences, row['mouse'] + '.h5')
    df = pd.read_hdf(fpath, '/metadataTrial/' + session)
    display(df.head())

,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,Reward,Go,End Recording,End Trial,Texture 7 P1200,Puff,No Go,Inappropriate Response,Puff1,DelayLength,TrialType
0,1,0.0000,0.0,1.2880,1.3010,3.1451,5.5163,7.5284,8.9885,8.9895,10.9026,11.6766,NaN,NaN,NaN,NaN,NaN,2.0121,Hit
0,2,15.8439,0.0,1.0270,NaN,2.8851,5.2573,NaN,NaN,NaN,NaN,10.2466,1.0410,5.5603,NaN,NaN,NaN,NaN,Early
0,3,30.2147,0.0,1.0200,NaN,2.8742,5.2453,7.2554,NaN,NaN,11.6837,12.3507,1.0301,NaN,9.5955,NaN,NaN,2.0101,CR
0,4,46.6906,0.0,1.0351,1.0461,2.8902,5.2613,7.2714,7.9875,7.9885,9.9006,10.6156,NaN,NaN,NaN,NaN,NaN,2.0101,Hit
0,5,61.4275,0.0,1.0180,1.0280,2.8721,5.2443,7.2544,7.8634,7.8644,9.7765,10.4936,NaN,NaN,NaN,NaN,NaN,2.0101,Hit


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,Reward,Go,End Recording,End Trial,Texture 7 P1200,No Go,Puff,Inappropriate Response,Puff1,No Response,DelayLength,TrialType
0,1,0.0000,0.0,1.0131,1.0231,2.8662,5.2363,7.2474,7.8655,7.8675,9.7766,10.5106,NaN,NaN,NaN,NaN,NaN,NaN,2.0111,Hit
0,2,14.6339,0.0,1.0170,NaN,2.8751,5.2463,7.2564,NaN,NaN,11.6906,12.3577,1.0280,9.6005,NaN,NaN,NaN,NaN,2.0101,CR
0,3,31.1198,0.0,1.0161,1.0271,2.8702,5.2403,7.2504,7.9685,7.9705,9.8806,10.6136,NaN,NaN,NaN,NaN,NaN,NaN,2.0101,Hit
0,4,45.8596,0.0,1.0201,NaN,2.8742,5.2443,7.2545,NaN,NaN,11.6837,12.3627,1.0301,9.5866,NaN,NaN,NaN,NaN,2.0102,CR
0,5,62.3506,0.0,1.0180,NaN,2.8761,5.2473,7.2574,NaN,NaN,11.7127,12.3477,1.0300,9.6185,NaN,NaN,NaN,NaN,2.0101,CR


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,Reward,Go,End Recording,End Trial,Texture 7 P1200,No Go,Puff,Inappropriate Response,Puff1,No Response,DelayLength,TrialType
0,1,0.0000,0.0,1.3131,1.3241,3.1662,5.5384,7.5495,8.1595,8.1615,10.0726,10.8197,NaN,NaN,NaN,NaN,NaN,NaN,2.0111,Hit
0,2,14.9729,0.0,1.0751,NaN,2.9332,5.3023,7.3134,NaN,NaN,11.7597,12.4107,1.0901,9.6716,NaN,NaN,NaN,NaN,2.0111,CR
0,3,31.5088,0.0,1.0131,1.0241,2.8692,5.2403,7.2505,7.8005,7.8025,9.7116,10.4376,NaN,NaN,NaN,NaN,NaN,NaN,2.0102,Hit
0,4,46.0687,0.0,1.0150,NaN,2.8711,5.2433,7.2534,NaN,NaN,11.6786,12.3277,1.0260,9.5865,NaN,NaN,NaN,NaN,2.0101,CR
0,5,62.5216,0.0,1.0161,1.0271,2.8712,5.2423,7.2524,7.9575,7.9585,9.8706,10.5916,NaN,NaN,NaN,NaN,NaN,NaN,2.0101,Hit


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,End Trial,Report,Reward,Go,End Recording,Texture 7 P1200,No Go,Puff,Inappropriate Response,Puff1,No Response,DelayLength,TrialType
0,1,0.0000,0.0,1.0290,1.040,2.8841,5.2553,11.6086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early
0,2,15.7349,0.0,1.0160,1.028,2.8721,5.2453,10.5346,7.2554,7.8844,7.8864,9.7935,NaN,NaN,NaN,NaN,NaN,NaN,2.0101,Hit
0,3,30.3907,0.0,1.0161,NaN,2.8722,5.2443,12.3157,7.2544,NaN,NaN,11.6827,1.0281,9.5936,NaN,NaN,NaN,NaN,2.0101,CR
0,4,46.8327,0.0,1.0190,1.029,2.8731,5.2483,10.4215,7.2594,7.7824,7.7844,9.6905,NaN,NaN,NaN,NaN,NaN,NaN,2.0111,Hit
0,5,61.3815,0.0,1.0120,NaN,2.8701,5.2403,12.3367,7.2504,NaN,NaN,11.6596,1.0260,9.5705,NaN,NaN,NaN,NaN,2.0101,CR


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,End Trial,Report,Reward,Go,End Recording,Texture 7 P1200,No Go,Puff,Inappropriate Response,Puff1,No Response,DelayLength,TrialType
0,1,0.0000,0.0,1.0451,1.0551,2.9002,5.2713,11.5367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early
0,2,15.6559,0.0,1.0161,1.0271,2.8712,5.2423,10.3666,7.2534,7.7194,7.7214,9.6285,NaN,NaN,NaN,NaN,NaN,NaN,2.0111,Hit
0,3,30.1467,0.0,1.0161,NaN,2.8742,5.2493,12.3247,7.2604,NaN,NaN,11.6857,1.0281,9.5946,NaN,NaN,NaN,NaN,2.0111,CR
0,4,46.5987,0.0,1.0190,1.0300,2.8741,5.2443,10.7646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early
0,5,61.4945,0.0,1.0141,1.0251,2.8692,5.2393,11.4007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Early


...

,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,Puff,Inappropriate Response,Puff1,End Recording,End Trial,No Go,Texture 7 P1200,Reward,Go,No Response,DelayLength,TrialType
0,1,0.0000,0.0,1.0151,1.0261,2.8722,5.2445,7.2597,8.1328,8.1618,8.1668,10.1970,11.8411,NaN,NaN,NaN,NaN,NaN,2.0152,FA
0,2,15.9726,0.0,1.0161,1.0261,2.8692,5.2385,7.2497,NaN,NaN,NaN,11.6741,12.3432,9.5829,NaN,NaN,NaN,NaN,2.0112,CR
0,3,32.4422,0.0,1.0151,NaN,2.8683,5.2385,7.2497,NaN,NaN,NaN,9.9890,10.7141,NaN,1.0251,8.0768,8.0788,NaN,2.0112,Hit
0,4,47.2827,0.0,1.0121,1.0241,2.8683,5.2385,7.2507,NaN,NaN,NaN,11.7092,12.3382,9.6159,NaN,NaN,NaN,NaN,2.0122,CR
0,5,63.7493,0.0,1.0181,1.0291,2.8733,5.2446,7.2548,8.2179,8.2479,8.2539,10.2831,11.9022,NaN,NaN,NaN,NaN,NaN,2.0102,FA


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,Puff,Inappropriate Response,Puff1,End Recording,End Trial,No Go,Texture 7 P1200,No Response,Reward,Go,DelayLength,TrialType
0,1,0.0000,0.0,1.0171,1.0281,2.8723,5.2445,7.2557,9.2209,9.2509,9.2539,11.2871,12.9383,NaN,NaN,NaN,NaN,NaN,2.0112,FA
0,2,17.0657,0.0,1.0221,1.0341,2.8773,5.2475,7.2577,NaN,NaN,NaN,11.6701,12.3062,9.5819,NaN,NaN,NaN,NaN,2.0102,CR
0,3,33.4993,0.0,1.0141,NaN,2.8713,5.2425,7.2527,NaN,NaN,NaN,11.4432,12.0942,NaN,1.0241,9.595,NaN,NaN,2.0102,Miss
0,4,49.7249,0.0,1.0151,1.0261,2.8713,5.2416,7.2538,9.3190,9.3490,9.3510,11.3842,13.0103,NaN,NaN,NaN,NaN,NaN,2.0122,FA
0,5,66.8617,0.0,1.0191,1.0301,2.8732,5.2445,7.2547,NaN,NaN,NaN,11.6971,12.3372,9.6009,NaN,NaN,NaN,NaN,2.0102,CR


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,No Go,End Recording,End Trial,Texture 7 P1200,Reward,Go,Puff,Puff1,Inappropriate Response,No Response,DelayLength,TrialType
0,1,0.0000,0.0,3.0491,3.0600,4.9035,7.2738,9.2852,11.6395,13.7302,14.3861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0114,CR
0,2,18.5157,0.0,1.0219,NaN,2.8767,5.2455,7.2563,NaN,10.0550,10.7369,1.0329,8.1432,8.1452,NaN,NaN,NaN,NaN,2.0108,Hit
0,3,33.3812,0.0,1.0489,1.0599,2.9037,5.2735,7.2843,NaN,11.5578,13.1747,NaN,NaN,NaN,9.4911,9.52,9.523,NaN,2.0108,FA
0,4,50.6825,0.0,1.0119,1.0249,2.8697,5.2414,7.2502,9.5950,11.6838,12.3137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0088,CR
0,5,67.1268,0.0,1.0179,NaN,2.8767,5.2485,7.3633,NaN,10.2960,11.0029,1.0299,8.3842,8.3872,NaN,NaN,NaN,NaN,2.1148,Hit


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,No Go,End Recording,End Trial,Texture 7 P1200,No Response,Reward,Go,Puff,Inappropriate Response,Puff1,DelayLength,TrialType
0,1,0.000,0.0,2.741,2.753,4.596,6.967,8.982,11.325,13.413,14.051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.015,CR
0,2,18.191,0.0,1.029,NaN,2.888,5.259,7.273,NaN,11.470,12.098,1.043,9.617,NaN,NaN,NaN,NaN,NaN,2.014,Miss
0,3,34.414,0.0,1.022,NaN,2.878,5.251,7.261,NaN,10.065,10.792,1.033,NaN,8.152,8.154,NaN,NaN,NaN,2.010,Hit
0,4,49.333,0.0,1.024,1.035,2.881,5.253,7.263,9.618,11.712,12.333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.010,CR
0,5,65.794,0.0,1.019,1.031,2.874,5.244,7.254,9.615,11.709,12.333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.010,CR


,Trial,Time,Begin Trial / Recording,Category Discrimination,Texture 1 P100,Stimulus,Delay,Report,Puff,Inappropriate Response,Puff1,End Recording,End Trial,No Go,Texture 7 P1200,Reward,Go,No Response,DelayLength,TrialType
0,1,0.0000,0.0,1.0221,1.0331,2.8773,5.2485,7.2637,8.0828,8.1128,8.1158,10.1510,11.7621,NaN,NaN,NaN,NaN,NaN,2.0152,FA
0,2,15.8926,0.0,1.0251,1.0361,2.8822,5.2555,7.2677,NaN,NaN,NaN,11.6741,12.3092,9.5799,NaN,NaN,NaN,NaN,2.0122,CR
0,3,32.3312,0.0,1.0191,1.0291,2.8733,5.2435,7.2537,NaN,NaN,NaN,11.7012,12.3642,9.6110,NaN,NaN,NaN,NaN,2.0102,CR
0,4,48.8239,0.0,1.0201,NaN,2.8772,5.2475,7.2577,NaN,NaN,NaN,9.9499,10.6580,NaN,1.0331,8.0378,8.0398,NaN,2.0102,Hit
0,5,63.6123,0.0,1.0231,1.0331,2.8763,5.2486,7.2588,NaN,NaN,NaN,11.6782,12.3343,9.5870,NaN,NaN,NaN,NaN,2.0102,CR


# 5. Fixing stuff
## 5.1 Add delay times

In [ ]:
prep.get_append_delay_times(pathPreferences)

## 5.2 Add active/passive

In [ ]:
prep.get_append_active_passive(pathPreferences)

# 6. Baseline subtraction

# 6.1 Plot stitched session, fit poly

**TODO**:
* Purge mismatching trial counts for trialStruct vs vid:
    - Go over data, delete last
* Very high variance of individual consecutive trials. May render entire idea useless
  - Explore more channels, mice, naive vs expert
  - Compare if Yaro has the same problem
  - Ask Yasir what he thinks about the origin of these fluctuations:
      - Real?
      - Motion artifacts?
      - Sth else?

In [ ]:
%matplotlib notebook
for idx, row in prep.dataPaths.iterrows():
    try: 
#         session = row['day'] + '_' + row['session']
        session="2017_03_29_session01"
        print(row['mouse'], session)
        
        times, dataRSP = prep.get_pooled_data_rel_times(pathPreferences, row['mouse'], session)
        prepcommon.example_poly_fit(times, dataRSP, ord=50, iCh=6)
    except:
        print('-- Failed')
    break

# 6.2 Process all sessions

In [ ]:
FPS = 20.0
iMin = int(0.0 * FPS)
iMax = int(1.0 * FPS)

In [ ]:
prep.crop_mismatch_trials(pathPreferences)

In [ ]:
# Test that this works with new metadata representation
prep.baseline_subtraction_dff(pathPreferences, iMin, iMax, overwrite=True)

In [ ]:
# Test that this works with new metadata representation
prep.baseline_subtraction_poly(pathPreferences, ord=15, overwrite=True)

# 7. Purification

### 7.1 Drop bad sessions

- mou_7 20171129_c
- mou_6 20171006_a

In [ ]:
prep.drop_preprocess_session(pathPreferences, 'mou_6', '20171006_a')

In [ ]:
prep.drop_preprocess_session(pathPreferences, 'mou_7', '20171129_c')

# 8. Sanity checking

## 8.1 Is reward part of video

Yes for all except MOU6

In [ ]:
prep.check_reward_in_data(pathPreferences)

# 9. Behaviour pooling

## 9.1 Movement Vectors

In [ ]:
sig2 = (0.05)**2 / 16
prep.behaviour_tune_resample_kernel('mou_5', '2017_03_06_session02', sig2,
                                    trialType='moveVect_100', trialIdx=3, srcFreq=30.0, trgFreq=20.0)

In [ ]:
prep.read_parse_behaviour(pathPreferences, skipExisting=False)